In [1]:
import sys
import os
import json
import pickle


def slashChanger(someString):
	tmp = ''
	for char in someString:
		if char == '\\':
			tmp += '/'
		else:
			tmp +=char
	return tmp

class fileRegister(object):
	"""docstring for fileRegister"""
	def __init__(self, arg):
		super(fileRegister, self).__init__()
		self.arg = arg




class singleShot(object):
	"""docstring for singleShot"""
	def __init__(self, id, pics_dir, json_dir, crop_dir, full_dir, spec_dir):
		super(singleShot, self).__init__()
		self.id       = id
		self.pics_dir = pics_dir
		self.json_dir = json_dir
		self.crop_dir = crop_dir
		self.full_dir = full_dir
		self.spec_dir = spec_dir
		self.crop_img_dir = None

		self.plot_img_full = None
		self.plot_img_crop = None
		self.plot_img_spec = None


		file = open(self.json_dir, 'r', encoding = 'utf-8')
		self.jsonDic = json.loads(file.read())
		file.close()

def refreshFolder(directory):
	directory = slashChanger(directory)
	print(directory)
	try:
		fnames = [s for s in os.listdir(directory) if os.path.isfile(directory + '/' + s) and '.jpeg' in s]
	except:
		return []
	print(fnames)
	fnames.sort()
	Fnames = [directory + '/' + s for s in fnames]

	mainThing = []
	for fname in fnames:
		tmp_pic_dir = directory + '/' + fname
		ID = fname.split('.')[0]
		tmp_json_dir = directory + '/' + ID + '.json' 		if os.path.isfile(directory + '/' + ID + '.json') else None
		tmp_crop_dir = directory + '/' + ID + '_crop.csv' 	if os.path.isfile(directory + '/' + ID + '_crop.csv') else None
		tmp_full_dir = directory + '/' + ID + '_full.csv' 	if os.path.isfile(directory + '/' + ID + '_full.csv') else None
		tmp_spec_dir = directory + '/' + ID + '_spec_NDVI.csv' if os.path.isfile(directory + '/' + ID + '_spec_NDVI.csv') else None
		mainThing.append(singleShot(ID, tmp_pic_dir, tmp_json_dir, tmp_crop_dir, tmp_full_dir, tmp_spec_dir))
	
	try:
		os.mkdir(directory + "/meta")
	except:
		pass

	with open(directory + '/meta/about.meta', 'wb') as config_dictionary_file:
		pickle.dump(mainThing, config_dictionary_file)
	return mainThing

def readMdata(directory):
	if (os.path.isfile(directory + '/meta/about.meta')):
		try:
			with open(directory + '/meta/about.meta', 'rb') as config_dictionary_file:
				mainThing = pickle.load(config_dictionary_file)
			return mainThing
		except:
			return None
	return refreshFolder(directory)
# refreshFolder(sys.aasdrgv[1])

In [1]:
import sys
import os
import json
import pickle
import shutil
import numpy as np
import pandas as pd
# class SingleShot(object):
#     """docstring for singleShot"""
#     def __init__(self, id, pics_dir, json_dir, crop_dir, full_dir, spec_dir):
#         super(singleShot, self).__init__()
#         self.id       = id
#         self.pics_dir = pics_dir
#         self.json_dir = json_dir
#         self.crop_dir = crop_dir
#         self.full_dir = full_dir
#         self.spec_dir = spec_dir
#         self.crop_img_dir = None

#         self.plot_img_full = None
#         self.plot_img_crop = None
#         self.plot_img_spec = None

#         if self.json_dir != None:
#             file = open(self.json_dir, 'r', encoding = 'utf-8')
#             self.jsonDic = json.loads(file.read())
#             file.close()
class SingleShot(object):
    """docstring for singleShot"""
    def __init__(self, id,
                 origRGB_dir, json_dir, rawBMPs_dirs, CSVfull_dir, 
                 CSVcrop_dir, CSVspec_dir, RGBwithPOV_dir, BMPwithPOV_dir):
        self.id       = id
        
        self.origRGB_dir      = origRGB_dir
        self.json_dir         = json_dir
        self.rawBMPs_dirs     = rawBMPs_dirs
        
        self.CSVcrop_dir    = CSVcrop_dir
        self.CSVfull_dir    = CSVfull_dir
        self.CSVspec_dir    = CSVspec_dir
        self.RGBwithPOV_dir = RGBwithPOV_dir
        self.BMPwithPOV_dir = BMPwithPOV_dir
        
        self.jsonDic = None
        if self.json_dir != None:
            if os.path.isfile(self.json_dir):
                file = open(self.json_dir, 'r', encoding = 'utf-8')
                self.jsonDic = json.loads(file.read())
                file.close()



In [73]:
class SetFormatter():
    def __init__(self, directory, typeDictionary = None):
        self.dir = directory
        self.dir_pcGen = os.path.join(self.dir, 'pcGen')
        self.Types = {}          #mostly associated with the downloading types
        self.usingTypes = {}     #mostly associated with the types you want to use in your work
        self.allFnames = {}
        self.setWhichTypes(typeDictionary)
        self.checkFiles()
        self.usingUniqueID_inSeperateBuckets = []
        self.numShots = 0
        self.shots = []
        self.ratio = pd.read_csv("ratio.csv")
        
    def setWhichTypes(self, tmpdict):
        self.Types['origRGB']    = tmpdict['origRGB']    if ("origRGB"    in tmpdict) else False
        self.Types['json']       = tmpdict['json']       if ("json"       in tmpdict) else False
        self.Types['rawBMPs']    = tmpdict['rawBMPs']    if ("rawBMPs"    in tmpdict) else False
        self.Types['CSVfull']    = tmpdict['CSVfull']    if ("CSVfull"    in tmpdict) else False
        self.Types['CSVcrop']    = tmpdict['CSVcrop']    if ("CSVcrop"    in tmpdict) else False
        self.Types['CSVspec']    = tmpdict['CSVspec']    if ("CSVspec"    in tmpdict) else False
        self.Types['RGBwithPOV'] = tmpdict['RGBwithPOV'] if ("RGBwithPOV" in tmpdict) else False
        self.Types['BMPwithPOV'] = tmpdict['BMPwithPOV'] if ("BMPwithPOV" in tmpdict) else False
        
    def dir_origRGB(self, ID):
        os.path.join(directory, '{}.jpg'.format(ID))
    def dir_json(self, ID):
        os.path.join(directory, '{}.json'.format(ID))
    def dir_rawBMPs(self, ID, i_th):
        os.path.join(directory, 'rawBMPs', '{}_{}.bmp'.format(ID,i_th))
    def dir_CSVfull(self, ID):
        os.path.join(directory, 'webGen', '{}_full.csv'.format(ID))
    def dir_CSVcrop(self, ID):
        os.path.join(directory, 'webGen', '{}_crop.csv'.format(ID))
    def dir_CSVspec(self, ID):
        os.path.join(directory, 'webGen', '{}_spec_NDVI.csv'.format(ID))
    def dir_RGBwithPOV(self, ID):
        os.path.join(directory, 'webGen', '{}.jpeg'.format(ID))
    def dir_BMPwithPOV(self, ID):
        os.path.join(directory, 'webGen', '{}.png'.format(ID))
    
    def checkFiles(self):
        self.allFnames['origRGB']    = [s for s in os.listdir(self.dir) if os.path.isfile(os.path.join(self.dir, s)) and '.jpg' in s]
        self.allFnames['json']       = [s for s in os.listdir(self.dir) if os.path.isfile(os.path.join(self.dir, s)) and '.json' in s]
        self.allFnames['rawBMPs']    = [s for s in os.listdir(os.path.join(self.dir,'rawBMPs')) if os.path.isfile(os.path.join(self.dir,'rawBMPs',s)) and '.bmp'           in s]
        self.allFnames['CSVfull']    = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '_full.csv'      in s]
        self.allFnames['CSVcrop']    = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '_crop.csv'      in s]
        self.allFnames['CSVspec']    = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '_spec_NDVI.csv' in s]
        self.allFnames['RGBwithPOV'] = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '.jpeg'          in s]
        self.allFnames['BMPwithPOV'] = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '.png'           in s]
    
    def cleanFiles(self):
        for s in os.listdir(self.dir):
            ss = os.path.join(self.dir, s)

            if os.path.isfile(ss):
                if not (s in self.allFnames['origRGB'] or 
                        s in self.allFnames['json']):
                    print("removed file:", s)
                    os.remove(ss)
            if os.path.isdir(ss):
                if (not s == "rawBMPs") and (not s == "webGen"):
                    shutil.rmtree(ss)
                    print("removed directory:", s)
                    
        tmpdir = os.path.join(self.dir, 'rawBMPs')
        for s in os.listdir(tmpdir):
            ss = os.path.join(tmpdir, s)
            if os.path.isfile(ss):
                if not (s in self.allFnames['rawBMPs']):
                    print("removed file: rawBMPs/", s)
                    os.remove(ss)                
            if os.path.isdir(ss):
                shutil.rmtree(ss)
                print("removed directory: rawBMPs/", s) 
        
        tmpdir = os.path.join(self.dir, 'webGen')
        for s in os.listdir(tmpdir):
            ss = os.path.join(tmpdir, s)
            if os.path.isfile(ss):
                if not (s in self.allFnames['CSVfull'] or 
                        s in self.allFnames['CSVcrop'] or 
                        s in self.allFnames['CSVspec'] or 
                        s in self.allFnames['RGBwithPOV'] or 
                        s in self.allFnames['BMPwithPOV']):
                    print("removed file: webGen/", s)
                    os.remove(ss)                
            if os.path.isdir(ss):
                shutil.rmtree(ss)
                print("removed directory: webGen/", s) 
    def read_ids(self, tmpdict, display = False):
        self.usingTypes['origRGB']    = tmpdict['origRGB']    if ("origRGB"    in tmpdict) else False
        self.usingTypes['json']       = tmpdict['json']       if ("json"       in tmpdict) else False
        self.usingTypes['rawBMPs']    = tmpdict['rawBMPs']    if ("rawBMPs"    in tmpdict) else False
        self.usingTypes['CSVfull']    = tmpdict['CSVfull']    if ("CSVfull"    in tmpdict) else False
        self.usingTypes['CSVcrop']    = tmpdict['CSVcrop']    if ("CSVcrop"    in tmpdict) else False
        self.usingTypes['CSVspec']    = tmpdict['CSVspec']    if ("CSVspec"    in tmpdict) else False
        self.usingTypes['RGBwithPOV'] = tmpdict['RGBwithPOV'] if ("RGBwithPOV" in tmpdict) else False
        self.usingTypes['BMPwithPOV'] = tmpdict['BMPwithPOV'] if ("BMPwithPOV" in tmpdict) else False
        
        self.usingUniqueID_inSeperateBuckets = []  #unique IDs over all set files (in seperate in their file types)
        if self.usingTypes['origRGB']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['origRGB']])
            print(("origRGB   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['json']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['json']])
            print(("json      :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['CSVfull']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('_')[0] + '_'+ fname.split('_')[1]  for fname in self.allFnames['CSVfull']])
            print(("CSVfull   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['CSVcrop']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('_')[0] + '_'+ fname.split('_')[1]  for fname in self.allFnames['CSVcrop']])
            print(("CSVcrop   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['CSVspec']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('_')[0] + '_'+ fname.split('_')[1]  for fname in self.allFnames['CSVspec']])
            print(("CSVspec   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['RGBwithPOV']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['RGBwithPOV']])
            print(("RGBwithPOV:unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['BMPwithPOV']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['BMPwithPOV']])
            print(("BMPwithPOV:unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['rawBMPs']:
            self.usingUniqueID_inSeperateBuckets.append(list(set([fname.split('_')[0] + '_'+ fname.split('_')[1]  for fname in self.allFnames['rawBMPs']])))
            print(("rawBMPs   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
            #self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['rawBMPs']])
            #since rawBMPs have multiple files in a single shot, and I want only ID's so I did following
        
        #now calculating unique IDs over all set of files
        print(("============================\n" if display else ''),  end='')

        self.usingUniqueIDs = []
        Tmp = len(self.usingUniqueID_inSeperateBuckets)
        for i in range(len(self.usingUniqueID_inSeperateBuckets[0])):
            tmpID = self.usingUniqueID_inSeperateBuckets[0][i]
            tmpbool = True
            for t in range(1,Tmp):
                tmpbool = tmpbool and (tmpID in self.usingUniqueID_inSeperateBuckets[t])
                if not tmpbool:
                    print(("- incomplete shot: {} has no {}\n".format(tmpID, list(tmpdict.keys())[t]) if display else ''),  end='')
                    break
            if tmpbool:
                self.usingUniqueIDs.append(tmpID)
           
        self.usingUniqueIDs.sort()
        self.numShots = len(self.usingUniqueIDs)
        print(("==============================================\nIn Given types, {} unique shots in this Set.\n".format(self.numShots) if display else ''),  end='')
        
        
        for i in range(self.numShots):
            tmpid       = self.usingUniqueIDs[i]
            tmp_dirorigRGB = os.path.join(self.dir,  '{}.jpg'.format(tmpid)) if self.usingTypes['origRGB']    else None
            tmp_dirjson    = os.path.join(self.dir, '{}.json'.format(tmpid)) if self.usingTypes['json']       else None
            tmp_dirrawBMPs = os.path.join(self.dir,'rawBMPs', '{}.jpeg'.format(tmpid)) if self.usingTypes['rawBMPs']    else None
            tmp_dirCSVfull = os.path.join(self.dir,'webGen',  '{}_full.csv'.format(tmpid)) if self.usingTypes['CSVfull']    else None
            tmp_dirCSVcrop = os.path.join(self.dir,'webGen',  '{}_crop.csv'.format(tmpid)) if self.usingTypes['CSVcrop']    else None
            tmp_dirCSVspec = os.path.join(self.dir,'webGen',  '{}_spec_NDVI.csv'.format(tmpid)) if self.usingTypes['CSVspec']    else None
            tmp_RGBwithPOV = os.path.join(self.dir,'webGen',  '{}.jpeg'.format(tmpid)) if self.usingTypes['RGBwithPOV'] else None
            tmp_BMPwithPOV = os.path.join(self.dir,'webGen',   '{}.png'.format(tmpid)) if self.usingTypes['BMPwithPOV'] else None
            self.shots.append(SingleShot(id             = tmpid,
                                         origRGB_dir    = tmp_dirorigRGB, 
                                         json_dir       = tmp_dirjson, 
                                         rawBMPs_dirs   = tmp_dirrawBMPs, 
                                         CSVfull_dir    = tmp_dirCSVfull, 
                                         CSVcrop_dir    = tmp_dirCSVcrop,
                                         CSVspec_dir    = tmp_dirCSVspec,
                                         RGBwithPOV_dir = tmp_RGBwithPOV,
                                         BMPwithPOV_dir = tmp_BMPwithPOV,
                                        ))

    def calcWGLFrom_webGen(self):
        if not (os.path.isdir(self.dir_pcGen)):
            os.mkdir(self.dir_pcGen)
        if self.numShots == 0:
            print("None shots in the folder")
            return
        self.setWave = np.zeros((self.numShots, 1,280))
        self.setGray = np.zeros((self.numShots, 1,280))
        self.setLeaf  = np.zeros((self.numShots,1,280))
        
        
        for i in range(self.numShots):
            tmpPdWeb = pd.read_csv(self.shots[i].CSVfull_dir)
            self.setWave[i,:]  = np.array(pd.read_csv(self.shots[i].CSVfull_dir)['WAVELENGTH'])
            self.setGray[i,:] = np.array(pd.read_csv(self.shots[i].CSVfull_dir)['GRAY'])
            self.setLeaf[i,:] = np.array(pd.read_csv(self.shots[i].CSVfull_dir)['LEAF'])
        
        np.save(os.path.join(self.dir_pcGen, 'setWave'), self.setWave)
        np.save(os.path.join(self.dir_pcGen, 'setGray'), self.setGray)
        np.save(os.path.join(self.dir_pcGen, 'setLeaf'), self.setLeaf)
        
    def calcWhiteFrom_webGen(self):
        self.setWhite  = np.zeros((self.numShots,1, 280))
        
        for i in range(self.numShots):
            self.setWhite[i,:] = (self.setGray[i,:]-10.5)/np.array(self.ratio['GRAY/WHITE'])+10.5
        
        np.save(os.path.join(self.dir_pcGen, 'setWhite'), self.setWhite)


In [74]:
Types = {
    "origRGB"   : True,
    "json"      : True,
    "rawBMPs"   : True,
    "CSVfull"   : True,
    "CSVcrop"   : True,
    "CSVspec"   : True,
    "RGBwithPOV": True,
    "BMPwithPOV": True,
}
sf = SetFormatter('/home/garid/myGit/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_', Types)
sf.read_ids(Types, display=True)

origRGB   :unique count: 326
json      :unique count: 326
CSVfull   :unique count: 326
CSVcrop   :unique count: 326
CSVspec   :unique count: 326
RGBwithPOV:unique count: 326
BMPwithPOV:unique count: 326
rawBMPs   :unique count: 326
- incomplete shot: 20191221_111708 has no json
In Given types, 325 unique shots in this Set.


In [75]:
sf.calcWGLFrom_webGen()
sf.calcWhiteFrom_webGen()

In [38]:
x = '/home/garid/myGit/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_/webGen/20191221_111630_full.csv'
a = pd.read_csv(x)

In [39]:
a

,PIXEL,WAVELENGTH,BG,GRAY,GRAY/WHITE,WHITE,LEAF,WHITE-BG,LEAF-BG,(LEAF-BG)/(WHITE-BG)
0,92,442.6,16.0,49.880000,0.5002,83.7329,184.200000,67.7329,168.2000,2.483284
1,93,444.8,16.0,50.173333,0.4979,84.6349,184.734286,68.6349,168.7343,2.458433
2,94,447.0,16.0,50.426667,0.4958,85.4366,185.328571,69.4366,169.3286,2.438607
3,95,449.2,16.0,50.580000,0.4941,85.9858,185.520000,69.9858,169.5200,2.422206
4,96,451.4,16.0,50.126667,0.4927,85.2646,185.474286,69.2646,169.4743,2.446766
...,...,...,...,...,...,...,...,...,...,...
275,367,1047.6,16.0,16.000000,0.9287,16.0000,16.000000,0.0000,0.0000,0.000000
276,368,1049.8,16.0,16.000000,0.9287,16.0000,16.000000,0.0000,0.0000,0.000000
277,369,1052.0,16.0,16.000000,0.9287,16.0000,16.000000,0.0000,0.0000,0.000000
278,370,1054.2,16.0,16.000000,0.9287,16.0000,16.000000,0.0000,0.0000,0.000000


In [14]:
type(a)

pandas.core.frame.DataFrame

In [25]:
np.array(a['GRAY']-16)/np.array(a['GRAY/WHITE'])- np.array(a['WHITE']-16)

array([ 6.83726510e-06,  3.33868909e-05,  7.79885042e-07,  3.28273629e-05,
       -3.55862187e-06,  3.67778682e-05,  7.06905996e-06, -4.11556276e-05,
        4.15949754e-05, -4.46269671e-05, -4.31453273e-05, -3.16477399e-06,
       -4.74141498e-05,  3.41072409e-05, -4.50540316e-05,  2.60030550e-05,
        1.04618802e-05,  3.19737858e-05, -4.94407152e-05, -3.98962284e-05,
        1.30801688e-05,  3.42218468e-05, -3.77518558e-05,  4.94790559e-05,
        2.76582144e-05,  2.36570608e-05,  4.13940560e-05, -1.33476081e-05,
       -2.35828158e-05, -6.44606789e-06,  9.04003443e-06, -3.22409077e-05,
       -9.76031095e-06, -4.56747405e-05, -4.21105825e-05, -3.17212470e-05,
        2.25600934e-05,  4.06601371e-05,  5.50952247e-06, -2.68225191e-05,
       -7.47227013e-06,  2.77911823e-05,  4.62169412e-06,  8.24893661e-06,
       -1.60564623e-05,  2.03269996e-05,  3.83480818e-05, -2.45847176e-05,
        2.15130031e-05,  1.82033090e-05,  2.62411347e-05,  1.93853435e-05,
       -3.55196100e-05, -

In [80]:
import numpy as np
country = np.array(['USA', 'Japan', 'UK', '', 'India', 'China']) 

In [81]:
country

array(['USA', 'Japan', 'UK', '', 'India', 'China'], dtype='<U5')

In [26]:
np.array(a['GRAY/WHITE'])

array([0.5002, 0.4979, 0.4958, 0.4941, 0.4927, 0.4916, 0.4904, 0.4892,
       0.4882, 0.487 , 0.4858, 0.4845, 0.4834, 0.4824, 0.4812, 0.4802,
       0.4792, 0.4781, 0.4768, 0.4754, 0.474 , 0.4727, 0.4715, 0.4703,
       0.4693, 0.4685, 0.4677, 0.467 , 0.4663, 0.4654, 0.4646, 0.4638,
       0.4631, 0.4624, 0.4618, 0.4611, 0.4604, 0.4595, 0.4586, 0.4577,
       0.4568, 0.4559, 0.4551, 0.4542, 0.4534, 0.4526, 0.452 , 0.4515,
       0.4512, 0.4512, 0.4512, 0.4512, 0.4513, 0.4514, 0.4516, 0.4519,
       0.4521, 0.4522, 0.4522, 0.4521, 0.4519, 0.4516, 0.4512, 0.4506,
       0.45  , 0.4493, 0.4484, 0.4476, 0.4468, 0.446 , 0.4454, 0.4448,
       0.4443, 0.4439, 0.4436, 0.4434, 0.4434, 0.4436, 0.4438, 0.4442,
       0.4447, 0.4452, 0.4458, 0.4465, 0.4472, 0.4478, 0.4485, 0.4492,
       0.4499, 0.4506, 0.4514, 0.4522, 0.4532, 0.4543, 0.4556, 0.4571,
       0.4588, 0.4607, 0.4629, 0.4653, 0.4679, 0.4706, 0.4737, 0.477 ,
       0.4808, 0.4848, 0.4889, 0.4932, 0.4979, 0.5027, 0.5077, 0.5129,
      

In [43]:
q = pd.DataFrame()

In [44]:
q['PIXEL']      = a['PIXEL']
q['WAVELENGTH'] = a['WAVELENGTH'] 
q['GRAY/WHITE'] = a['GRAY/WHITE']


,Unnamed: 0,PIXEL,WAVELENGTH,GRAY/WHITE
0,0,92,442.6,0.5002
1,1,93,444.8,0.4979
2,2,94,447.0,0.4958
3,3,95,449.2,0.4941
4,4,96,451.4,0.4927
...,...,...,...,...
275,275,367,1047.6,0.9287
276,276,368,1049.8,0.9287
277,277,369,1052.0,0.9287
278,278,370,1054.2,0.9287
